<div >
    <img src = "../Banner.jpg" />
</div>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ignaciomsarmiento/MALM/blob/main/06_Rec_Usuarios/L06_Colab.ipynb)

# Sistemas de Recomendación

## ¿Qué son los sistemas de recomendaciones?


Las preferencias de los individuos suelen seguir patrones que los sistemas de recomendación pueden aprovechar, por ejemplo,

- Si te interesó:  <div style="max-width:200px">
<img src = "figs/iron_man.jpg" />
</div>



- También te puede interesar: <div style="max-width:200px">
<img src = "figs/thor.jpg" />
</div> 




<div  style="max-width: 80%;">
<img src = "figs/item_jeans.png"/>
</div>



<div  style="max-width: 80%;">
<img src = "figs/usuario_libros.png"/>
</div>

- Los sistemas de recomendación entonces encuentran patrones que son utilizados para predecir qué otros productos podrían gustarnos y generar sugerencias, de forma tal que  los usuarios encuentren contenido atractivo en un gran corpus. 

- Estos sistemas son muy exitosos, por ejemplo según un estudio del 2013 de [McKinsey](https://www.mckinsey.com/industries/retail/our-insights/how-retailers-can-keep-up-with-consumers), el 35% de los artículos comprados en Amazon surgen de estos sistemas de recomendación, y por lo tanto vale la pena estudiarlos cuidadosamente.





<div style="max-width:500px">
<img src = "figs/BING.png" />
</div>


## Introducción: el problema de predicción 

Para entender un poco mejor cuál es el problema al que nos enfrentamos, supongamos que tenemos una matriz con $m$ usuarios y $n$ ítems. En esta matriz cada fila representa un usuario y cada columna un ítem. El valor de la celda denota el rating que le dió cada usuario al ítem. Este valor lo denotamos como $r_{ij}$ que será entonces el rating que le dio el usuario $i$ al ítem $j$. Un ejemplo de esta matriz sería la siguiente:

<div  style="max-width: 40%;">
<img src = "figs/fig1.png"/>
</div>

Esta matriz tiene 7 usuarios ($m=7$) y 6 ítems ($n=6$). El usuario 1 le otorgó al ítem 1 un rating de 4, por lo tanto $r_{11}=4$. Sin embargo, no todos los usuarios calificaron/utilizaron todos los ítems y por lo tanto no todos tienen ranking. En el caso del usuario 1 este no utilizó los ítems 3, 4, y 6; por lo tanto aparecen con un signo de interrogación. Estos datos están faltando en la matriz.


Consideremos otro ejemplo más concreto. Imaginemos que somos una compañía como Netflix y tenemos un repositorio de 20.000 películas y 5.000 usuarios. Tenemos además un sistema que registra la calificación que cada usuario le otorga a una película en particular. Es decir, tenemos una  matriz de tamaño 5,000 × 20,000. Sin embargo, es muy probable que los usuarios sólo habrán visto sólo una fracción de las películas (difícilmente todos vieron 20.000 películas!). Por lo tanto, la matriz será poco densa, la mayoría de las entradas en la matriz estarán vacías.


El problema de predicción, tiene por objetivo predecir estos valores faltantes utilizando toda la información que tenemos a disposición: las calificaciones registradas, datos sobre películas, datos sobre usuarios, etc. Si el sistema es capaz de predecir con precisión los valores que faltan, podrá dar excelentes recomendaciones. Por ejemplo, si el usuario $i$ no ha utilizado el ítem $j$, pero nuestro sistema predice una calificación muy alta, es entonces muy probable que le guste $j$ si lo descubre a través del sistema de recomendación.

## Filtrado Colaborativo Basado en Usuarios.

El filtrado colaborativo aprovecha el poder de la colaboración para generar recomendaciones. 

 <div style="max-width:400px">
<img src = "figs/Colab.png" />
</div> 



### Ejemplo

Para entender un poco mejor cuál es el problema al que nos enfrentamos, supongamos que tenemos una matriz con 5 usuarios y 5 productos, en este caso restaurantes. 

El valor de la celda denota el rating que le dió cada usuario al restaurante. Este valor lo denotamos como $r_{ij}$ que será entonces el rating que le dio el usuario $i$ al restaurante $j$. 
 

In [ ]:
#cargamos librerias
import pandas as pd
import numpy as np

# Cargamos y visualizamos  los datos
ratings = pd.read_csv('https://github.com/ignaciomsarmiento/datasets/raw/refs/heads/main/Ratings.csv')
ratings.head()

In [ ]:
# pivotamos la tabla para tener una matriz de usuarios y restaurantes
r_matrix = ratings.pivot_table(values='rating', index='user_id', columns='restaurant_id')
r_matrix.head()

In [ ]:
# creamos una matriz de usuarios y restaurantes donde remplazamos los valores nulos por 0
r_matrix_dummy = r_matrix.copy().fillna(0)
r_matrix_dummy

### Filtrado colaborativo sencillo (simple): medias, y medias ponderadas.


#### Medias 

- Esta estrategia consiste en calcular el rating promedio que le asignó cada usuario 

In [ ]:
 puntuac_promedio = ratings.groupby('restaurant_id').mean()
 pd.DataFrame(puntuac_promedio.iloc[:,1])

#### Medias ponderadas

- Podemos hacer algo más sofisticado usando recomendaciones de usuarios similares?


- Podemos hacer una media ponderada


$$
r_{ur}=\frac{\sum_{u',u'\neq u}sim(u,u').r_{u'r}}{\sum_{u',u'\neq u}sim(u,u')}
$$



- Es decir la predicción del rating del usuario $u$ para el restaurante $r$, $r_{ur}$, es la suma ponderada de los ratings de los otros usuarios ($u'$) a este restaurante, 


- Ponderado por cuán similares son los usuarios $u'$ a $u$. 


- Como medimos similitud?

    - Existen múltiples medidas de distancia que se utilizan para medir la similitud. 

    - La distancia de coseno, que suele ser la más utilizada en los sistemas de recomendación.

    - Matemáticamente

$$
coseno(x,y)=\frac{x.y'}{|x||y|}
$$


Es decir, es el cociente del producto punto, dividido por las normas de los vectores.

<center>
<img src = "figs/dist_cos.png" alt = "coseno" style = "width: 300px;"/>
</center>


- Si el ángulo es 0  de grados, entonces los vectores se solapan, y el coseno es igual a 1 
- Si el ángulo es 90 de grados, los vectores forman un angulo recto, y el coseno es igual a 0.
- Si el ángulo es 180 de grados, los vectores estan en sentido opuesto, y  el coseno es igual a -1.



In [ ]:
import math

math.cos(math.radians(0))

**¿Qué medida de similitud elegir?** 

Notemos que, si dos vectores, $x$ e $y$, tienen media cero, el coseno del angulo entre ambos coincidirá con el coeficiente de correlación de Pearson.

La distancia del coseno suele ser una medida de similitud ampliamente utilizada en los sistemas de recomendación por varias razones:

   - *Independencia de magnitud*: La distancia del coseno mide la similitud entre dos vectores basándose en el ángulo entre ellos, en lugar de depender de las magnitudes de los vectores. Esto es útil en los sistemas de recomendación, donde las magnitudes de las características (por ejemplo, la calificación de un usuario para un artículo) pueden variar significativamente entre diferentes usuarios o elementos. Al ser independiente de la magnitud, el coseno permite capturar patrones de similitud más robustos.

   - *Espacios de alta dimensión*: En los sistemas de recomendación, las características que representan usuarios o elementos a menudo se representan como vectores en espacios de alta dimensión. En estos espacios, la noción de distancia euclidiana puede volverse menos significativa debido al fenómeno conocido como "maldición de la dimensionalidad". La distancia del coseno puede ser más adecuada en estos casos porque se centra en la orientación relativa de los vectores, lo que permite medir la similitud de manera más efectiva.

   - *Eficiencia computacional*: Calcular la distancia del coseno es computacionalmente eficiente en comparación con otras métricas de similitud más complejas, como la distancia de Mahalanobis o la distancia euclidiana en espacios de alta dimensión. Esto es importante en sistemas de recomendación que manejan grandes conjuntos de datos y deben realizar cálculos rápidos para generar recomendaciones en tiempo real.

   - *Contexto de recomendación*: En muchos sistemas de recomendación, las preferencias de los usuarios pueden estar dispersas, lo que significa que solo han interactuado con una pequeña parte del conjunto total de elementos. La distancia del coseno tiende a funcionar bien en estos escenarios, ya que ignora las diferencias entre características no observadas, lo que puede conducir a una mejor generalización y recomendaciones más precisas.

A pesar de estas ventajas, es importante destacar que no existe un enfoque único para todos los sistemas de recomendación. La elección de la medida de similitud depende en gran medida del contexto y la naturaleza de los datos en un sistema específico. Por ejemplo,

 - En caso que las magnitudes sean importantes, la distancia euclidiana es una medida apropiada para utilizar. 
 - Si la correlación es lo que importa, entonces la correlación de Pearson o el coseno son más apropiadas. 
 - Si la popularidad del ítem bajo estudio es importante, entonces el producto escalar será una medida adecuada. Puesto que si los ítems aparecen con mucha frecuencia (por ejemplo, videos populares de YouTube) estos tienden a normas grandes y el producto escalar capturará mejor esta información. Sin embargo, si no somos cuidadosos, aquellos artículos populares terminarán siendo los más recomendados. En casos como este, se puede definir una medida que "regularice" las normas, como por ejemplo: $similitud(x,y)=|x|^\alpha  |y|^\alpha cos(x,y)$ para algún $\alpha\in(0,1)$.
 

##### Creamos una función recomendadora usando esto

In [ ]:
# Importamos cosine_similarity 
from sklearn.metrics.pairwise import cosine_similarity

#Calculamos la similitud de coseno 
cosine_sim = cosine_similarity(r_matrix_dummy, r_matrix_dummy)

# Transformamos la matriz de similitud en un DataFrame
cosine_sim = pd.DataFrame(cosine_sim, index=r_matrix.index, columns=r_matrix.index)

cosine_sim


In [ ]:
?cosine_similarity

In [ ]:
# Definimos una función para calcular el promedio ponderado de los ratings
def cf_user_wmean(user_id, restaurant_id):
    
    # Primero verificamos si el restaurante está en la matriz
    if restaurant_id in r_matrix:
    
        #Buscamos las medidas de similitud con los otros usuarios
        sim_scores = cosine_sim[user_id]
        
        # Obtenemos los ratings no faltantes de la matriz bajo evaluación
        m_ratings = r_matrix[restaurant_id]
        
         # Obtenemos los índices de los restaurantes sin rating 
        idx = m_ratings[m_ratings.isnull()].index
        
        # Nos quedamos con similitudes y ratings completos
        sim_scores = sim_scores.drop(idx)
        m_ratings = m_ratings.dropna()
        
        # Calculamos la media ponderada
        wmean_rating = np.dot(sim_scores, m_ratings)/ sim_scores.sum()
    
    else:
        # Si no tenemos ninguna información retornamos 3
        wmean_rating = 3.0
    
    return wmean_rating

In [ ]:
cf_user_wmean(1,1)

In [ ]:
sim_scores = cosine_sim[1]
sim_scores

In [ ]:
m_ratings =r_matrix[1]
m_ratings

In [ ]:
idx = m_ratings[m_ratings.isnull()].index
idx

Falta entonces 

$$
r_{um}=\frac{\sum_{u',u'\neq u}sim(u,u').r_{u'm}}{\sum_{u',u'\neq u}sim(u,u')}
$$


In [ ]:
sim_scores1 = sim_scores.drop(idx)

m_ratings = m_ratings.dropna()
        
# Calculamos la media ponderada
wmean_rating = np.dot(sim_scores1, m_ratings)/ sim_scores1.sum()
wmean_rating

## Un ejemplo más realista y complicado


Para ilustrar cómo construir e implementar este tipo de sistema de recomendación utilizaremos nuevamente los datos de  [MovieLens](https://grouplens.org/datasets/movielens/latest/) provista abiertamente por [grouplens](https://grouplens.org/about/what-is-grouplens/) para: **"avanzar la teoría y la práctica de la computación social mediante la construcción y la comprensión de sistemas *(de recomendación)* utilizados por personas reales".**

Esta versión de los datos contiene varias bases con información de los usuarios, de las películas, y de los ratings. Tendremos que combinar estas bases para obtener una matriz similar a la ilustrada en la gráfica anterior. Carguemos entonces las librerías y los datos de usuarios:

In [ ]:
# Cargamos las librerías a utilizar
import pandas as pd
import numpy as np
# Cargamos los datos de los usuarios
u_cols = ['user_id', 'edad', 'genero', 'ocupacion', 'codigo_postal']

users = pd.read_csv('https://raw.githubusercontent.com/ignaciomsarmiento/RecomSystemsLectures/main/L04_Usuarios/data/u.user', sep='|', names=u_cols,
 encoding='latin-1')
users.head()

Podemos ver que esta base contiene un identificador de usuario, su edad, género, ocupación, y el código postal donde viven. Luego cargamos la base de películas:

In [ ]:
# Cargamos los datos de las películas
i_cols = ['movie_id','titulo', 'fecha_estreno', 'fecha_estreno_video', 'URL_IMDb', 'desconocido', 'Accion', 'Aventura',
  'Animacion', 'Infantil', 'Comedia', 'Crimen', 'Documental', 'Drama', 'Fantasia',
  'Cine-Noir', 'Horror', 'Musical', 'Misterio', 'Romance', 'Ciencia_ficcipn', 'Thriller', 'Guerra', 'Western']

movies = pd.read_csv('https://github.com/ignaciomsarmiento/datasets/raw/refs/heads/main/u.item', sep='|', names=i_cols, encoding='latin-1')

movies.head()

Esta base contiene un identificador de película, el título, cuándo fue estrenada en el cine, cuándo fue estrenada en video, la dirección web a IMDb, y el género al que pertenece, incluyendo una columna que marca como desconocido si el género no pertenece a ninguna de las restantes. Notemos además que una película puede pertenecer a varios géneros, por ejemplo: Toy Story pertenece  al genero Animación e Infantil. Pero por ahora nos importa sólo el identificador y el título de la película, por lo que nos quedamos con estas columnas:

In [ ]:
## (Completar) Tip:  columnas de interes 'movie_id', 'titulo'


La última base que necesitamos cargar es la de ratings:

In [ ]:
# Cargamos los datos de los ratings 
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']

ratings = pd.read_csv('https://github.com/ignaciomsarmiento/datasets/raw/refs/heads/main/u.data', sep='\t', names=r_cols,
 encoding='latin-1')

ratings.head()

Esta base tiene información sobre qué `rating` el usuario (`user_id`) le otorgó a cada película (`movie_id`), e información de cuándo fue que hizo tal calificación (`timestamp`). Esta última columna no la utilizaremos por lo que la excluiremos de la base.

In [ ]:
# Quitamos la columna timestamp 


Con estos elementos podemos construir la matriz que vincule usuarios, películas y ratings. Para ello usamos la función `pivot_table`.

In [ ]:
## (Completar) Tip: usar pivot_table siendo valores rating y las columnas movie_id
r_matrix = ratings.pivot_table(values='rating', index='user_id', columns='movie_id')

r_matrix.head()

In [ ]:
## (Completar) Tip:  la dimen con comando shape


Esta matriz entonces vincula los usuarios con las películas. Las filas denotan los 943 usuarios y las columnas las 1682 películas en la base. El usuario 1 le otorgó a la película 1 un rating de 5, por lo tanto $r_{11}=5$. Al igual que en el ejemplo anterior, no todos los usuarios calificaron/vieron todas las películas y por lo tanto no todas tienen ranking. En el caso del usuario 1 este no calificó las películas 1673 a 1682 y por lo tanto aparecen con `NaN`; estos datos están faltando en la matriz.

La tarea es entonces buscar estrategias para completar estos datos faltantes.

### Medias

Comencemos por la estrategia quizás más simple e intuitiva para completar las celdas faltantes. Esta estrategia consiste en calcular el rating promedio que le asignó cada usuario que ranquearon esta película. No hacemos distinción entre los usuarios, y el rating de cada uno recibirá el mismo peso. Habrá casos donde ninguna de las películas ha sido ranqueada, en tales situaciones le pondremos un rating default de 3. 

Para evaluar además la performance de esta estrategia, dividiremos la base original en bases de entrenamiento y prueba.

y definiremos como métrica el error cuadrático medio

Con la base lista podemos construir nuestra función recomendadora:

In [ ]:
## (Completar) Tip:  funcion de media


Podemos entonces evaluar el RMSE que obtendremos de la base de prueba de haber entrenado la función recomendadora en la base de entrenamiento:

In [ ]:
## (Completar) Tip: calcular el RMSE

Podemos entonces utlizar este resultado como una base para la comparación a los modelos subsiguientes.

### Medias ponderadas


In [ ]:
# Rellenamos los faltantes con 0
r_matrix_dummy = r_matrix.copy().fillna(0)

In [ ]:
r_matrix_dummy

Con esto estamos en condiciones de usar la función `cosine_similarity` que va a calcular la similitud de coseno.

In [ ]:
# Importamos cosine_similarity 
from sklearn.metrics.pairwise import cosine_similarity

#Calculamos la similitud de coseno 
cosine_sim = cosine_similarity(r_matrix_dummy, r_matrix_dummy)

# Transformamos la matriz resultante en un dataframe
cosine_sim = pd.DataFrame(cosine_sim, index=r_matrix.index, columns=r_matrix.index)

cosine_sim.head(10)

Tenemos entonces como resultado una matriz que muestra la similitud de coseno entre los diferentes usuarios. La matriz diagonal que muestra $1$ nos dice que la similitud entre el usuario 1 y sí mismo es 1. Note que el 0.16 dice que la similitud entre 1 y 2 es de 0.16. En este caso, dado que estamos evaluando ratings positivos, esta matriz retornará entradas positivas.

Con esta matriz calculada podemos entonces utilizarla para construir nuestra función recomendadora basada en medias ponderadas.

In [ ]:
## (Completar) Tip:  Crear función recomendadora basada en medias con input user_id y movie_id


In [ ]:
## (Completar) Tip: Probar la funcion


Estamos en condiciones entonces de evaluar este modelo:

In [ ]:
## (Completar) Tip: evaluar

Notemos que esta simple "sofisticación" mejora el desempeño predictivo.

### Datos demográficos de los usuarios

El modelo anterior entonces incorporó  similitud entre las calificaciones de los usuarios para mejorar las recomendaciones. ¿Qué sucede entonces si utilizamos variables demográficas? La intuición es que individuos de similares géneros, profesiones, edades, etc., tendrán preferencias más similares que individuos con características demográficas diferentes.

En este caso lo que haremos es restringir a los ratings de individuos con las mismas características demográficas, en jerga estadística, utilizaremos un modelo de medias condicionales.

Para ello, primero unimos los datos originales sobre usuarios con nuestra base de entrenamiento.

In [ ]:
merged_df = pd.merge(X_train, users)

In [ ]:
merged_df.head()

Con esta base que tiene ahora variables demográficas podemos intentar mejorar nuestro recomendador. Ilustremos entonces una función recomendadora basada en género y ocupación. Para ello, primero calculamos el rating promedio por género y ocupación. En `pandas` esto es relativamente sencillo utilizando la función `groupby`.

In [ ]:
users = users.set_index('user_id')

In [ ]:
# Calculamos el rating promedio basado en género y ocupación
# Completar
gen_occ_mean = merged_df[['genero', 'rating', 'movie_id', 'ocupacion']].pivot_table(
    values='rating', index='movie_id', columns=['ocupacion', 'genero'], aggfunc='mean')

gen_occ_mean.head()

Tenemos entonces el promedio de rating por género y ocupación. Así, los administradores femeninos ranquearon la película 1 con un promedio de 3.93 y los masculinos con 3.75. Mientras que los artistas no ranquearon la película 2. Con esta información generamos una función recomendadora que incorpore esta información y en los casos donde no tenemos información utilizaremos el rating 3 como default.

In [ ]:
## (Completar) Tip: generar las recomendaciones basadas en género y ocupación

Evaluamos entonces esta función recomendadora:

In [ ]:
## (Completar) Tip: Evaluar el modelo de filtrado colaborativo basado en medias por género y ocupación

Vemos que esta función es la que peor funciona de las antes evaluadas. La función que incorporaba el coseno como medida de similitud lograba mejores recomendaciones. En la sección siguiente nos centraremos en entender mejor las medidas de similitud y cómo estas nos permiten mejorar los recomendadores.

# Referencias

- Banik, R. (2018). Hands-on recommendation systems with Python: start building powerful and personalized, recommendation engines with Python. Packt Publishing Ltd.

- Covington, P., Adams, J., & Sargin, E. (2016). Deep Neural Networks for YouTube recommendations. Proceedings of the 10th ACM Conference on Recommender Systems. https://doi.org/10.1145/2959100.2959190 

- Google developers. (n.d.). Recommendation systems. Google. Consultado en Abril 3, 2022. Disponible en https://developers.google.com/machine-learning/recommendation/overview 

- Google developers. (n.d.). Embeddings: Motivation From Collaborative Filtering. Consultado en Mayo 13, 2022. Disponible en  https://developers.google.com/machine-learning/crash-course/embeddings/motivation-from-collaborative-filtering
